# Omeka S API - Sample bulk upload scripts

These scripts were created based on the code in the individual item notebook,
with changes and code creation by claude.ai in December 2024.


In [4]:
import requests
import csv
import json
import os

In [2]:
def import_omeka_items(csv_path, media_directory, site_api, item_set_id, key_identity, key_credential):
    """
    Import items and their associated media files to an Omeka S site via API.
    
    Parameters:
    - csv_path: Path to the CSV file containing item metadata
    - media_directory: Directory containing media files
    - site_api: Base URL of the Omeka S API
    - item_set_id: ID of the Omeka S item set to add items to
    - key_identity: API key identity
    - key_credential: API key credential
    
    Returns:
    - List of response statuses for each item import
    """
    # API endpoint for items
    action = '/items'
    
    # Prepare API authentication parameters
    params = {
        'key_credential': key_credential,
        'key_identity': key_identity,
    }
    
    # Store import results
    import_results = []
    
    # Read the CSV file
    with open(csv_path, 'r', encoding='utf-8') as csvfile:
        # Assume the first row contains headers
        reader = csv.DictReader(csvfile)
        
        # Process each row in the CSV
        for row_index, row in enumerate(reader):
            # Prepare files dictionary for this item
            files = {}
            
            # Construct item metadata
            item_metadata = {
                "@type": "o:Item",
                "o:item_set": [{"o:id": item_set_id}],
                "o:media": []
            }
            
            # Add title (adjust field name as needed)
            if 'title' in row:
                item_metadata['dcterms:title'] = [{
                    "property_id": 1,
                    "property_label": "Title",
                    "@value": row['title'],
                    "type": "literal"
                }]
            
            # Add other metadata fields dynamically (customize as needed)
            for key, value in row.items():
                # Skip empty values and special fields
                if key.lower() == 'title' or not value:
                    continue
                
                # You might need to map CSV column names to Omeka property IDs
                # This is a placeholder and should be customized based on your specific metadata schema
                item_metadata[f'dcterms:{key}'] = [{
                    "property_id": 1,  # This should be replaced with correct property ID
                    "property_label": key.capitalize(),
                    "@value": value,
                    "type": "literal"
                }]
            
            # Find and add media files
            # Assumes there's a unique identifier in the CSV that matches part of the media filename
            media_files = []
            for filename in os.listdir(media_directory):
                # Customize the matching logic based on your file naming convention
                if any(identifier.lower() in filename.lower() for identifier in row.values()):
                    media_path = os.path.join(media_directory, filename)
                    media_files.append(media_path)
            
            # Prepare media files for upload
            for file_index, media_file_path in enumerate(media_files):
                files[f'file[{file_index}]'] = open(media_file_path, 'rb')
                
                # Add media metadata
                item_metadata['o:media'].append({
                    "o:ingester": "upload",
                    "file_index": str(file_index),
                    "o:item": {},
                    "dcterms:title": [{
                        "property_id": 1,
                        "property_label": "Title",
                        "@value": f"Media file for {row.get('title', 'item')}",
                        "type": "literal"
                    }]
                })
            
            # Add item metadata to files
            files['data'] = (None, json.dumps(item_metadata))
            
            try:
                # Send API request to create item
                r = requests.post(site_api + action, params=params, files=files)
                import_results.append({
                    'row': row_index,
                    'status_code': r.status_code,
                    'url': r.url,
                    'response': r.text
                })
                
                # Close any open file handles
                for file_obj in files.values():
                    if hasattr(file_obj, 'close'):
                        file_obj[1].close()
            
            except Exception as e:
                import_results.append({
                    'row': row_index,
                    'error': str(e)
                })
    
    return import_results

In [ ]:
# Example usage
if __name__ == '__main__':
    results = import_omeka_items(
        csv_path='path/to/your/metadata.csv',
        media_directory='path/to/media/files',
        site_api='http://your-omeka-s-site/api',
        item_set_id=100, # input your set ID here, if desired
        key_identity='key-identity-for-your-API-user-agent',
        key_credential='key-credential-for-your-API-user-agent'
    )
    
    # Print import results
    for result in results:
        print(result)

In [5]:
# usage in practice
if __name__ == '__main__':
    results = import_omeka_items(
        csv_path='../collection-project/collection_items_data.csv',
        media_directory='../collection-project/item-files',
        site_api='http://jajohnst.si676.si.umich.edu/omeka-s/api',
        item_set_id=654,
        key_identity='yAu5NAEEtJCvGx4s8XgJSgvTFjwvLD6j',
        key_credential='nj0K6JAqnFgeqyP3Kq40SOhl6fiPBXky'
    )
    
    # Print import results
    for result in results:
        print(result)

{'row': 0, 'status_code': 200, 'url': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/items?key_credential=nj0K6JAqnFgeqyP3Kq40SOhl6fiPBXky&key_identity=yAu5NAEEtJCvGx4s8XgJSgvTFjwvLD6j', 'response': '{"@context":"http:\\/\\/jajohnst.si676.si.umich.edu\\/omeka-s\\/api-context","@id":"http:\\/\\/jajohnst.si676.si.umich.edu\\/omeka-s\\/api\\/items\\/655","@type":"o:Item","o:id":655,"o:is_public":true,"o:owner":{"@id":"http:\\/\\/jajohnst.si676.si.umich.edu\\/omeka-s\\/api\\/users\\/1","o:id":1},"o:resource_class":null,"o:resource_template":null,"o:thumbnail":null,"o:title":"Carnegie Library, Cordele, Georgia","thumbnail_display_urls":{"large":null,"medium":null,"square":null},"o:created":{"@value":"2024-12-07T15:34:44+00:00","@type":"http:\\/\\/www.w3.org\\/2001\\/XMLSchema#dateTime"},"o:modified":{"@value":"2024-12-07T15:34:44+00:00","@type":"http:\\/\\/www.w3.org\\/2001\\/XMLSchema#dateTime"},"o:primary_media":null,"o:media":[],"o:item_set":[{"@id":"http:\\/\\/jajohnst.si676.si.umich.e